# Start a Simulated SNMP Agent

In [1]:
import subprocess

try:
    if process:
        pass
except:
    process = subprocess.Popen(
        [
            'snmpsimd.py',
            '--agent-udpv4-endpoint=127.0.0.1:1161',
            '--agent-udpv6-endpoint=[::1]:1161'
        ],
        stdout=subprocess.DEVNULL,
        stderr=subprocess.DEVNULL,
        shell=False
    )

# Create a DataFrame mimicking a database

In [1]:
from uuid import UUID

import pandas as pd

df = (
    pd.DataFrame(
        data=[
            [UUID('3a8083c9-79ff-4ed9-969c-e204cee391b3'), 'host1', 'localhost:1161', 'recorded/linux-full-walk', 'DNS Resolution'],
            #[UUID('83f73383-c7e6-44ff-a063-8c79f339777b'), 'host2', '127.0.0.1', 'recorded/linux-full-walk', 'Timeout'],
            [UUID('3cae774a-ec98-4679-b20a-dc1947b0546f'), 'host3', '[::1]:1161', 'recorded/linux-full-walk', 'IPv6'],
            [UUID('3cae774a-ec98-4679-b20a-dc1947b0546f'), 'host3', '[::1]:1161', 'recorded/linux-full-walk', 'Preserved Index'],
        ],
        columns=['id', 'hostname', 'ip_address', 'community_string', 'notes']
    )
    .set_index(['id'])
)
df

,hostname,ip_address,community_string,notes
id,,,,
3a8083c9-79ff-4ed9-969c-e204cee391b3,host1,localhost:1161,recorded/linux-full-walk,DNS Resolution
3cae774a-ec98-4679-b20a-dc1947b0546f,host3,[::1]:1161,recorded/linux-full-walk,IPv6
3cae774a-ec98-4679-b20a-dc1947b0546f,host3,[::1]:1161,recorded/linux-full-walk,Preserved Index


# Construct a MIB-like Definition of the SNMP Objects to Collect

In [2]:
import numpy as np

from snmp_fetch.var_bind import VarBind

# definitions are built in an abbreviated hierarchy similiar to a MIB

class InterfaceTable(VarBind):
    oid='.1.3.6.1.2.1'

class IfTable(VarBind):
    parent = InterfaceTable
    oid='.2.2.1'
    index=np.dtype([('if_index', np.uint64)])
    
    def before_pivot(df):
        return df.set_index('if_index')

class IfAdminStatus(VarBind):
    parent = IfTable
    oid='.7'
    dtype=np.dtype([('admin_status', np.uint64)])
    
    def before_pivot(df):
        df['admin_status'] = df['admin_status'].astype(pd.UInt64Dtype())
        return df

class IfOperStatus(VarBind):
    parent = IfTable
    oid='.8'
    dtype=np.dtype([('oper_status', np.uint64)])
    
    def before_pivot(df):
        df['oper_status'] = df['oper_status'].astype(pd.UInt64Dtype())
        return df

class IfXTable(VarBind):
    """Test"""
    parent = InterfaceTable
    oid='.31.1.1.1'
    index=np.dtype([('if_index', np.uint64)])
    
    def before_pivot(df):
        return df.set_index('if_index')

class IfXAlias(VarBind):
    """Test"""
    parent = IfXTable
    oid='.1'
    dtype=np.dtype([('alias', 'S256')])
    
    def before_pivot(df):
        df['alias'] = df['alias'].str.decode('utf-8', errors='ignore')
        return df

# InterfaceTable.describe()

InterfaceTable.null_var_binds()

[([1, 3, 6, 1, 2, 1, 2, 2, 1, 7], (88, 8)),
 ([1, 3, 6, 1, 2, 1, 2, 2, 1, 8], (88, 8)),
 ([1, 3, 6, 1, 2, 1, 31, 1, 1, 1, 1], (96, 256))]

In [3]:
from snmp_fetch import PduType, SnmpConfig, fetch, capi

results0, errors0 = capi.fetch(
    PduType.BULKGET,
    [(i, h, c) for i, (h, c) in enumerate(df[['ip_address', 'community_string']].values)],
    InterfaceTable.null_var_binds()
)

# Display Errors

In [4]:
errors0

[]

# Display Results

In [6]:
from functools import reduce
from operator import attrgetter
from snmp_fetch.var_bind import concatv_dtypes, dtype_array
from snmp_fetch.fp.maybe import Maybe, Just
from functools import partial



index = df.index.names
df0 = df.reset_index()
df0.index = df0.index.set_names(['#index'])

InterfaceTable.to_pandas(results0, data=df0, index=index)

,if_index,admin_status,oper_status,alias,#timestamp,hostname,ip_address,community_string,notes
id,,,,,,,,,
3a8083c9-79ff-4ed9-969c-e204cee391b3,1,1,1,lo,2019-11-06 22:34:42+00:00,host1,localhost:1161,recorded/linux-full-walk,DNS Resolution
3a8083c9-79ff-4ed9-969c-e204cee391b3,2,1,1,eth0,2019-11-06 22:34:42+00:00,host1,localhost:1161,recorded/linux-full-walk,DNS Resolution
3cae774a-ec98-4679-b20a-dc1947b0546f,1,1,1,lo,2019-11-06 22:34:42+00:00,host3,[::1]:1161,recorded/linux-full-walk,IPv6
3cae774a-ec98-4679-b20a-dc1947b0546f,2,1,1,eth0,2019-11-06 22:34:42+00:00,host3,[::1]:1161,recorded/linux-full-walk,IPv6
3cae774a-ec98-4679-b20a-dc1947b0546f,1,1,1,lo,2019-11-06 22:34:42+00:00,host3,[::1]:1161,recorded/linux-full-walk,Preserved Index
3cae774a-ec98-4679-b20a-dc1947b0546f,2,1,1,eth0,2019-11-06 22:34:42+00:00,host3,[::1]:1161,recorded/linux-full-walk,Preserved Index


# Stop the Simulated SNMP Agent

In [ ]:
process.kill()
process.communicate()
del process

In [22]:
oid_dtype

Just(value=dtype(('<u8', (10,))))

In [23]:
index_dtype

dtype([('if_index', '<u8')])